In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import math
import numpy as np
from functools import reduce

In [2]:
n =  2  # dimension of input
m =  3  # dimension of output
batch =  4  # batch size

In [3]:
def calc_logit(input, weight, bias=None):
    """
    Calculate logit = input.mm(weight.t()) + bias

    Args:
        input:  [1 x n]         Input vector
        weight:  [m x n]        Weight matrix
        bias:  [m]              Bias vector

    Returns:
        logit: [1 x n]          Logit = input.mm(weight.t()) + bias

    """

    logit = input.mm(weight.t())
    if bias is not None:
        logit += bias.unsqueeze(0).expand_as(logit)

    return logit

## ESGD

In [4]:
def esgd_arctan(grad_output, input, weight, bias, output):

    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]
    grad_weight = grad_output_scaled.t().mm(input)  # [m x n]
    grad_bias = grad_output_scaled.sum(0).squeeze(0)  # [m]

    return grad_input, grad_weight, grad_bias

## Old ISGD implementation

In [100]:
def calc_backwards_variables(saved_tensors, logit, grad_output, lr, mu):
    """
    Calculate the variables required for back-propagation
    
    Args:
        saved_tensors:          Stores from forward-propagation the input, weight, bias, output
        logit: [1 x n]          Stores from forward-propagation the logit
        grad_output: [1 x m]    The gradient that has been back-propagated to this layer
        lr: [1]                 Learning rate
        mu: [1]                 Ridge-regularization constant

    Returns:
        input: [1 x n]          Input vector
        weight: [m x n]         Weight matrix
        bias: [m]               Bias vector
        output [1 x m]          Input to the next layer = logit put through the non-linear activation function
        logit: [1 x n]          Logit
        s: [1 x m]              Sign of back-propagated gradient
        z_norm: [1]             2-norm of (input, 1)
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        c: [1 x m]              Logit contracted by ridge-regularization
    """

    # Unpack saved values
    input, weight, bias, output = saved_tensors

    # ISGD constants
    s = torch.sign(grad_output)
    z_norm = math.sqrt((torch.norm(input) ** 2 + 1.0))
    d = z_norm / math.sqrt(1.0 + lr * mu) * torch.sqrt(torch.abs(grad_output))
    c = logit / (1.0 + lr * mu)

    return input, weight, bias, output, logit, s, z_norm, d, c

In [99]:
def calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias):
    """
    Calculate the gradient of the weight matrix and bias vector

    Args:
        weight: [m x n]         Weight matrix
        mu: [1]                 Ridge-regularization constant
        lr: [1]                 Learning rate
        a: [1 x m]              Solution of ISGD update
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        input: [1 x n]          Input vector
        z_norm: [1]             2-norm of (input, 1)
        bias: [m]               Bias vector

    Returns:
        grad_weight: [m x n]    Gradient of the weight matrix
        grad_bias: [m]          Gradient of the bias vector

    """

    grad_weight = weight * mu / (1.0 + lr * mu) - (a * d).t().mm(input) / z_norm ** 2
    grad_bias = bias * mu / (1.0 + lr * mu) - (a * d).squeeze() / z_norm ** 2
    return grad_weight, grad_bias

In [97]:
def real_root_closest_to_zero(coeff):
    roots = np.roots(coeff)
    real_roots = [root.real for root in roots if root.imag == 0]
    root_closest_to_zero = reduce((lambda x, y: x if (abs(x) < abs(y)) else y), real_roots)
    return root_closest_to_zero

In [98]:
def isgd_arctan(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.00000001, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
    coeff = np.array([((lr * d) ** 2).numpy()[0],
                      (2 * lr * d * c).numpy()[0],
                      (c ** 2 + 1).numpy()[0],
                      (s * d).numpy()[0]])

    root_closest_to_zero = np.apply_along_axis(real_root_closest_to_zero, 0, coeff)
    a = torch.from_numpy(root_closest_to_zero).unsqueeze(1).t().type(torch.FloatTensor)

    # Calculate grad_weight, grad_bias and return all gradients
    grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]

    return grad_input, grad_weight, grad_bias

In [96]:
def isgd_arctan2(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.0, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
    d_d = d.double()
    s_m_d = (s * d).double()
    c_d = c.double()

    a = d_d * 0  # [b x m]
    a_diff = 1  # Norm difference between previous and current a values
    iter_count = 0  # Count of number of a iterations
    while a_diff > 1e-15:
        a_new = - s_m_d / (1.0 + (lr * d_d * a + c_d) ** 2)  # [b x m]
        a_diff = torch.norm(a - a_new)
        a = a_new  # [b x m]
        iter_count += 1
        if iter_count >= 50:
            assert (iter_count < 50), 'Arctan update has failed to converge'

    # Make a float so that can be operated on with other tensors
    a = a.float()

    # Calculate grad_weight, grad_bias and return all gradients
    grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [b x m]
    grad_input = grad_output_scaled.mm(weight)  # [b x n]

    return grad_input, grad_weight, grad_bias

## New ISGD implementation

In [158]:
def cube_solver(b, c):


    
#     # Check that the solution is valid
#     diff = x * (1 + (x - c)**2) - b
#     print(diff)

    return v #.float()

In [283]:
def isgd_new_arctan(input, weight, bias, output, logit, grad_output):
    
    # Hyperparameters
    lr = 0.0001
    mu = 0.0

    # ISGD constants
    # Make doubles to avoid rounding errors
    z_norm_squared = (torch.norm(input, p=2, dim=1) ** 2 + 1.0).double()  # [b]
    c = (logit / (1.0 + lr * mu)).double()  # [b x m]
    g = grad_output.double()
    b = torch.mul(z_norm_squared, g.t()).t() * lr / (1 + lr * mu)
    
    # Solves for v in the equation:  v * (1 + (v - c)**2) = b
    delta = 27 * (b ** 2) - 4 * b * (c ** 3) - 36 * b * c + 4 * (c ** 4) + 8 * (c ** 2) + 4  # [b x m]
    gamma = 27 * b - 2 * (c ** 3) - 18 * c  # [b x m]
    beta = (3 * ((3 * delta) ** (1/2)) + gamma)
    cr_beta = torch.sign(beta) * (torch.abs(beta) ** (1/3))
    v = cr_beta / (3 * (2 ** (1/3))) - (2 ** (1/3)) * (3 - c**2) / (3 * cr_beta) + 2 * c / 3
    
    # Calculate u in terms of v
    u = torch.div(v.t(), lr * z_norm_squared).t()
    u = u.float()

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [b x m]
    grad_input = grad_output_scaled.mm(weight)  # [b x n]

    # Calculate grad_weight, grad_bias
    grad_weight = weight * mu / (1.0 + lr * mu) + u.t().mm(input)  # [m x n]
    grad_bias = bias * mu / (1.0 + lr * mu) + u.t().sum(1)  # [m]
    
    return grad_input, grad_weight, grad_bias

## Test the differences between ESGD, ISGD old and ISGD_new

In [284]:
# Random data
grad_output = torch.randn(batch, m)     # [b x m]
input = torch.randn(batch, n)           # [b x n]
weight = torch.randn(m, n)          # [m x n]
bias = torch.randn(m,)              # [m]

# Forward propagation
# Calculate logit [1 x m], where logit = input.mm(weight.t()) + bias
logit = calc_logit(input, weight, bias)

# Non-linear activation function
output = torch.atan(logit)  # [1 x m]

# Calculate gradients
esgd_grads = esgd_arctan(grad_output, input, weight, bias, output)
isgd_grads = isgd_arctan2(grad_output, input, weight, bias, output, logit)
isgd_new_grads = isgd_new_arctan(input, weight, bias, output, logit, grad_output)

# print(esgd_grads)
# print(isgd_new_grads)
# Print difference
# print('Difference between ESGD and ISGD old')
# print([(x-y) for x,y in zip(isgd_grads, esgd_grads)])

print('\nDifference between ESGD and ISGD new')
print([(x-y) for x,y in zip(isgd_new_grads, esgd_grads)])


Difference between ESGD and ISGD new
[
 0  0
 0  0
 0  0
 0  0
[torch.FloatTensor of size 4x2]
, 
1.00000e-04 *
 -1.1265  1.7729
  2.4188  0.4476
  0.2325  0.0131
[torch.FloatTensor of size 3x2]
, 
1.00000e-04 *
 -1.8495
  0.6923
  0.0529
[torch.FloatTensor of size 3]
]


In [166]:
# Hyperparameters
lr = 0.00000001
mu = 0.0

# ISGD constants
z_norm_squared = (torch.norm(input, p=2, dim=1) ** 2 + 1.0).double()  # [b]
c = (logit / (1.0 + lr * mu)).double()  # [b x m]

# Calculate u
# Calculate conditions for u

#     b = torch.mul(z_norm_squared, grad_output.t()).t() * lr / (1 + lr * mu)
#     v = cube_solver(b, c)
#     u = torch.div(v.t(), lr * z_norm_squared).t()

g = grad_output.double()

b = torch.mul(z_norm_squared, g.t()).t() * lr / (1 + lr * mu)
v = cube_solver(b, c)
u = torch.div(v.t(), lr * z_norm_squared).t()

<class 'torch.DoubleTensor'>


In [168]:
print('v_diff: ', v_diff(g, c, lr, z_norm_squared, v))
print('u_diff: ', u_diff(g, c, lr, z_norm_squared, u))

v_diff:  
1.00000e-15 *
 -0.0467  0.2271  0.1269
  0.0436 -0.0494 -0.0723
 -0.0755  0.6545 -0.0094
  2.1662 -0.1020 -0.0920
[torch.DoubleTensor of size 4x3]

u_diff:  
1.00000e-08 *
 -0.0851  0.4141  0.2314
  0.3963 -0.4494 -0.6574
 -0.4769  4.1371 -0.0593
  7.7471 -0.3648 -0.3290
[torch.DoubleTensor of size 4x3]



In [151]:
def v_diff(grad_output, c, lr, z_norm_squared, v):
    b = torch.mul(z_norm_squared, grad_output.t()).t() * lr / (1 + lr * mu)
    return v - b / (1.0 + (c - v) ** 2)

In [7]:
def u_diff(grad_output, c, lr, z_norm_squared, u):
    return u - grad_output / (1.0 + (c - lr * torch.mul(z_norm_squared, u.t()).t()) ** 2) / (1.0 + lr * mu)

In [114]:
b = torch.mul(z_norm_squared, grad_output.t()).t() * lr / (1 + lr * mu)
print(v - b / (1.0 + (c - v) ** 2))
u = torch.div(v.t(), lr * z_norm_squared).t()
# print(u - torch.div((b / (1.0 + (c - v) ** 2) ).t(), lr * z_norm_squared).t())
print(u - grad_output / (1.0 + (c - v) ** 2) / (1 + lr * mu) )
print(u)


1.00000e-11 *
  0.0000  0.0000  0.0000
  0.0000  0.3638 -0.3638
  0.0000  1.4552  0.3638
  0.0000  0.0000 -0.0909
[torch.FloatTensor of size 4x3]


1.00000e-08 *
  0.0000  0.1863  0.0000
  2.9802  0.0000 -2.9802
  0.0000  5.9605  0.0000
  0.0000 -2.9802 -0.7451
[torch.FloatTensor of size 4x3]


-0.0437  0.0156  0.0824
-0.3537 -0.3011 -0.3721
 0.8572  0.6016 -0.2674
 0.3294 -0.4565 -0.0494
[torch.FloatTensor of size 4x3]



In [184]:
# If accurate these should all be zero
print('explicit u_diff: ', u_diff(g, c, lr, z_norm_squared, u_esgd.double()))
print('implicit u_diff: ', u_diff(g, c, lr, z_norm_squared, u))

explicit u_diff:  
 0.0892 -0.9234 -0.7942
 1.8232  0.7002 -0.5315
 0.0309 -0.3597 -0.2075
 0.1497 -0.8657 -0.1209
[torch.DoubleTensor of size 4x3]

implicit u_diff:  
1.00000e-08 *
 -0.0851  0.4141  0.2314
  0.3963 -0.4494 -0.6574
 -0.4769  4.1371 -0.0593
  7.7471 -0.3648 -0.3290
[torch.DoubleTensor of size 4x3]



In [177]:
b = torch.mul(z_norm_squared, g.t()).t() * lr / (1 + lr * mu)
print(v - b / (1.0 + (c - v) ** 2))
print(torch.div(v.t(), lr * z_norm_squared).t() - torch.div((b / (1.0 + (c - v) ** 2)).t(), lr * z_norm_squared).t())
print(u - g / (1.0 + (c - lr * torch.mul(z_norm_squared, u.t()).t()) ** 2) / (1.0 + lr * mu))


1.00000e-15 *
 -0.0467  0.2271  0.1269
  0.0436 -0.0494 -0.0723
 -0.0755  0.6545 -0.0094
  2.1662 -0.1020 -0.0920
[torch.DoubleTensor of size 4x3]


1.00000e-08 *
 -0.0851  0.4141  0.2314
  0.3963 -0.4494 -0.6574
 -0.4769  4.1371 -0.0593
  7.7471 -0.3648 -0.3290
[torch.DoubleTensor of size 4x3]


1.00000e-08 *
 -0.0851  0.4141  0.2314
  0.3963 -0.4494 -0.6574
 -0.4769  4.1371 -0.0593
  7.7471 -0.3648 -0.3290
[torch.DoubleTensor of size 4x3]



In [291]:
a = torch.Tensor([1,0,3]) / torch.Tensor([1,0,3])

In [296]:
torch.sum(a != a)

1

In [298]:
b = torch.Tensor([[1,0,3], [1,0,3]])
torch.sum(b)

8.0

In [300]:
0.0 ** (1/2)

0.0